In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [94]:
import pandas as pd
import numpy as np
import calendar
import datetime
import re
import os

def filter_data(input_file_path, output_file_path,keyword,affirm_report,order_report_table,reg_report):
    current_date=datetime.datetime.now()
    file = os.listdir(input_file_path)[0]
    dataFrame= pd.read_csv(os.path.join(input_file_path, file))
    # select rows containing text "Affirm"
    data_filtered = dataFrame.loc[dataFrame['DESCRIPTION'].str.contains(keyword),['DATE', 'DESCRIPTION', 'RECEIVED']]

    #split the decription
    new =data_filtered.DESCRIPTION.str.split(expand=True)
    data_filtered['DEPOSIT']=new[15]
    data_filtered.drop(columns =["DESCRIPTION"], inplace = True)
    data_filtered['DESCRIPTION']=new[3]+' '+new[4]
  
    # split the deposit id
    data_filtered[['IID','DEPOSIT_ID']]=data_filtered.DEPOSIT.str.split(":",expand=True)
    data_filtered.drop(columns =["DEPOSIT"], inplace = True)
    data_filtered.drop(columns =["IID"], inplace = True)

      
    #read the affirm report file
    data_report=pd.read_csv(affirm_report)
    data_report.rename(columns = {'deposit_id':'DEPOSIT_ID'}, inplace = True) 
    new_data_report=data_report.drop_duplicates(subset='order_id') #drop duplicate based on order id
    #combine two csv files based on DEPOSIT ID
    df = pd.merge(data_filtered, new_data_report,on="DEPOSIT_ID")
    # print(df.to_string())
    df.rename(columns = {'RECEIVED':'BANK AMOUNT'}, inplace = True)
    df.rename(columns = {'order_id':'ORDER'}, inplace = True)
    df['FEE']=df['fees']+df['txn_fees']
    df['ORDER']=df['ORDER'].astype(str)

    #read the order report table
    report_table=pd.read_csv(order_report_table)
    report_table.rename(columns = {'Order_Number':'ORDER'}, inplace = True)
    #covert the object type to float
    report_table['Item_Price']=report_table['Item_Price'].apply(lambda x: float(x.split()[0].replace(',', '')))
    report_table['Item_Discount']=report_table['Item_Discount'].apply(lambda x: float(x.split()[0].replace(',', '')))
    
    #combine report table to df
    data_table=pd.merge(df,report_table,on='ORDER')
    data_table.rename(columns = {'Item_Price':'PRICE'}, inplace = True) # rename columns
    data_table.rename(columns = {'Item_Discount':'DISCOUNT'}, inplace = True) # rename columns
    data_table.rename(columns = {'Order_Date':'QBO DATE'}, inplace = True) # rename columns
    data_table['QBO Description']=data_table['ORDER']
    data_table['INCOME']=data_table['PRICE']

    #add new columns
    data_table['PRICE']=data_table['PRICE'].astype(float)
    data_table['DISCOUNT']=data_table['DISCOUNT'].astype(float)
    data_table['NET AMOUNT']=data_table['PRICE']+data_table['FEE']+data_table['DISCOUNT']
    
    #add a new columns check by comparing two columns
    data_table['CHECK']=np.where((data_table['NET AMOUNT'] == data_table['total_settled']), 'Correct', 'Incorrect')

    #change the datatype to int
    data_table['ORDER']=data_table['ORDER'].astype(int)

    #compare the result with registration report to get coursemonth
    registr_report=pd.read_csv(reg_report)
    data_table['Success'] = data_table['ORDER'].isin(registr_report['Order id']).astype(int)
    registr_report.rename(columns = {'Order id':'ORDER'}, inplace = True) # rename columns
    registr_report.rename(columns = {'Session Name':'SESSION'}, inplace = True) # rename columns

    #split the session Name to get the course month
    registr_report[['cl1','cl2','Month']]=registr_report.SESSION.str.split("|",expand=True)
    registr_report[['colmn0','colmn1','colmn2','Course Month','clmn4']]=registr_report.Month.str.split(expand=True)
    
    registr_report['Course Month']= pd.to_datetime(registr_report['Course Month'])
    registr_report['Account'] = 'N/A'
    registr_report.loc[registr_report['Course Month'] < current_date, 'Account'] = '4000 Tution Income'
    registr_report.loc[registr_report['Course Month'] > current_date, 'Account'] = '2200 Unearned Revenue'

    registr_report['Course Month']=registr_report['Course Month'].dt.strftime('%b %Y') 
    
    
    #trying to merge the files
    new_df=pd.merge(data_table,registr_report,on='ORDER',how='left')
    new_df['Course Month']=new_df['Course Month'].astype(object)
    new_df['Course Month']=np.where((new_df['Success'] == 1), new_df['Course Month'], 'N/A')
    new_df.loc[new_df['Course Month'] == 'N/A', 'Account'] = '4000 Tution Income'
    df2 =new_df.groupby("BANK AMOUNT").size().reset_index(name='counts')
    df_sum=new_df.groupby("BANK AMOUNT").sum()
    # df_sum['Bankamount']=df_sum['BANK AMOUNT']
    # new_amount =df_sum.Bankamount.str.split('$',expand=True)
    df_sum['DrN']=df_sum.FEE.abs()+df_sum['DISCOUNT']
    new_dflist=pd.merge(new_df,df_sum,on='BANK AMOUNT',how='left')
    # new_dflist['BANK AMOUNT']=new_dflist['BANK AMOUNT'].astype(str)
    new_dflist2=new_dflist['BANK AMOUNT'].str.split('$',n = 1,expand=True)
    new_dflist['Bankamount']=new_dflist2[1].apply(lambda x: float(x.split()[0].replace(',', '')))
    new_df['Credit(Income/Revenue)']=new_dflist['DrN']+new_dflist['Bankamount']
    # print(new_dflist.to_string())
    #create ne df from new_df with required colmn
    output_data=new_df[['DATE', 'DESCRIPTION','BANK AMOUNT','PRICE','FEE','DISCOUNT','NET AMOUNT','CHECK','Course Month','ORDER','QBO DATE','QBO Description','INCOME','Account','Credit(Income/Revenue)']].copy()
    # print(output_data.dtypes)
    print(output_data.to_string())
    #convert the output to csv file
    output_data.to_csv('affirm_report_nsummary.csv', encoding = 'utf-8-sig') 
    report='affirm_report_nsummary.csv'
    # output_data.to_csv(os.path.join(output_file_path,report), index = False) 
    
    

In [ ]:
filter_data('/content/drive/MyDrive/Colab Notebooks/input_file_path','/content/drive/MyDrive/Colab Notebooks/output_file_path','AFFIRM','/content/drive/MyDrive/Affirm Testing/Affirm_Report_0123.csv','/content/drive/MyDrive/Affirm Testing/order_report_Jan22_Jan23.csv','/content/drive/MyDrive/Colab Notebooks/registration_report_Dec22 _Jan23  - Sheet1.csv')
